In [4]:
import os
for split in ('train','test'):
    print(split.upper())
    for cls in os.listdir(os.path.join(BASE_DIR, split)):
        path = os.path.join(BASE_DIR, split, cls)
        print(f"  {cls}: {len(os.listdir(path))} files")

TRAIN
  DAMAGED_SIGNS: 18 files
  NORMAL_SIGNS: 29 files
TEST
  DAMAGED_SIGNS: 18 files
  NORMAL_SIGNS: 29 files


In [6]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint

# ─── 1) Directory setup ────────────────────────────────────────────────────────
BASE_DIR   = r'C:\Users\ADMIN\OneDrive\Desktop\AI_PROJECT\SIGN'
TRAIN_DIR  = os.path.join(BASE_DIR, 'train')
TEST_DIR   = os.path.join(BASE_DIR, 'test')

# ─── 2) Hyperparameters ────────────────────────────────────────────────────────
IMG_SIZE   = (150, 150)
BATCH_SIZE = 16
EPOCHS     = 15

# ─── 3) Data generators ────────────────────────────────────────────────────────
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.1
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)
test_gen = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

# ─── 4) Sanity checks ───────────────────────────────────────────────────────────
print(f"→ {train_gen.samples} training images across {train_gen.num_classes} classes")
print(f"→ {test_gen.samples}   testing images across {test_gen.num_classes} classes")

if train_gen.samples == 0:
    raise ValueError(f"No images found in TRAIN_DIR: {TRAIN_DIR}")
if test_gen.samples == 0:
    raise ValueError(f"No images found in TEST_DIR:  {TEST_DIR}")

# ─── 5) Build the simple CNN ───────────────────────────────────────────────────
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(*IMG_SIZE, 3)),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

# ─── 6) Train & save best weights ──────────────────────────────────────────────
checkpoint = ModelCheckpoint(
    'sign_cnn_best.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max'
)

history = model.fit(
    train_gen,
    epochs=EPOCHS,
    validation_data=test_gen,
    callbacks=[checkpoint]
)

# ─── 7) Load best weights & evaluate ───────────────────────────────────────────
model.load_weights('sign_cnn_best.h5')
loss, acc = model.evaluate(test_gen, verbose=1)
print(f"\nSign damage detection test accuracy: {acc * 100:.2f}%")


Found 47 images belonging to 2 classes.
Found 47 images belonging to 2 classes.
→ 47 training images across 2 classes
→ 47   testing images across 2 classes


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,828,481 (18.42 MB)

 Trainable params: 4,828,481 (18.42 MB)

...

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 352ms/step - accuracy: 0.4130 - loss: 0.8739 - val_accuracy: 0.6170 - val_loss: 0.6860
Epoch 3/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 292ms/step - accuracy: 0.6718 - loss: 0.6646 - val_accuracy: 0.6170 - val_loss: 0.6430
Epoch 4/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 316ms/step - accuracy: 0.5663 - loss: 0.6906 - val_accuracy: 0.6170 - val_loss: 0.6395
Epoch 5/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 312ms/step - accuracy: 0.6366 - loss: 0.6467 - val_accuracy: 0.6170 - val_loss: 0.6487
Epoch 6/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 298ms/step - accuracy: 0.5943 - loss: 0.6861 - val_accuracy: 0.6170 - val_loss: 0.6406
Epoch 7/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 294ms/step - accuracy: 0.6004 - loss: 0.6438 - val_accuracy: 0.6170 - val_loss: 0.6331
Epoch 8/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 298ms/step - accuracy: 0.6180 - loss: 0.6544 - val_accuracy: 0.6170 - val_loss: 0.6266
Epoch 9/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step - accuracy: 0.6742 - loss: 0.6317

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 362ms/step - accuracy: 0.6705 - loss: 0.6340 - val_accuracy: 0.7021 - val_loss: 0.6208
Epoch 10/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 314ms/step - accuracy: 0.7261 - loss: 0.6165 - val_accuracy: 0.6809 - val_loss: 0.6154
Epoch 11/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 315ms/step - accuracy: 0.7553 - loss: 0.6364 - val_accuracy: 0.6809 - val_loss: 0.6107
Epoch 12/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 295ms/step - accuracy: 0.7366 - loss: 0.5778 - val_accuracy: 0.6809 - val_loss: 0.6109
Epoch 13/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - accuracy: 0.7340 - loss: 0.5896

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 345ms/step - accuracy: 0.7261 - loss: 0.5936 - val_accuracy: 0.7447 - val_loss: 0.5935
Epoch 14/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step - accuracy: 0.7409 - loss: 0.5580

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 348ms/step - accuracy: 0.7259 - loss: 0.5659 - val_accuracy: 0.7872 - val_loss: 0.5682
Epoch 15/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - accuracy: 0.7292 - loss: 0.5550

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 339ms/step - accuracy: 0.7331 - loss: 0.5659 - val_accuracy: 0.8085 - val_loss: 0.5447
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.7402 - loss: 0.5921

Sign damage detection test accuracy: 80.85%
